In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, EarlyStoppingCallback
from datasets import load_dataset, concatenate_datasets, Dataset
import torch
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import os

from utils import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

In [2]:
def one_hot_encode(y, nb_classes=2):
    if not isinstance(y, np.ndarray):
        y = np.expand_dims(np.array(y), 0)
    res = np.eye(nb_classes)[np.array(y).reshape(-1)]
    return res.reshape(list(y.shape)+[nb_classes])[0]

def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True, max_length=250)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average=None)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1.mean(),
        'precision': precision.mean(),
        'recall': recall.mean()
    }

def acc_at_k(y_true, y_pred, k=2):
    y_pred = torch.tensor(y_pred) if type(y_pred) != torch.Tensor else y_pred
    y_true = torch.tensor(y_true) if type(y_true) != torch.Tensor else y_true
    total = len(y_true)
    y_weights, y_idx = torch.topk(y_true, k=k, dim=-1)
    out_weights, out_idx = torch.topk(y_pred, k=k, dim=-1)
    correct = torch.sum(torch.eq(y_idx, out_idx) * y_weights)
    acc = correct / total
    if acc.item() > 1:
        print(y_true.shape, y_true)
        print(y_pred.shape, y_pred)
    return acc.item()

def CEwST_loss(logits, target, reduction='mean'):
    """
    Cross Entropy with Soft Target (CEwST) Loss
    :param logits: (batch, *)
    :param target: (batch, *) same shape as logits, each item must be a valid distribution: target[i, :].sum() == 1.
    """
    logprobs = torch.nn.functional.log_softmax(logits.view(logits.shape[0], -1), dim=1)
    batchloss = - torch.sum(target.view(target.shape[0], -1) * logprobs, dim=1)
    if reduction == 'none':
        return batchloss
    elif reduction == 'mean':
        return torch.mean(batchloss)
    elif reduction == 'sum':
        return torch.sum(batchloss)
    else:
        raise NotImplementedError('Unsupported reduction mode.')

def compute_metrics_w_soft_target(pred):
    labels = pred.label_ids
    preds = pred.predictions
    acc = acc_at_k(labels, preds, k=2)
    return {
        'accuracy': acc,
    }

class Trainer_w_soft_target(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs[0]
        loss = CEwST_loss(logits, labels)
        if return_outputs:
            return loss, outputs
        return loss
    
class DefaultCollator:
    def __init__(self):
        pass
    def __call__(self, batch):
        return torch.utils.data.dataloader.default_collate(batch)

In [3]:
# ['bert-base-uncased', 'roberta-base', 'xlnet-base-cased']
# ['ORIG', 'INV', 'SIB', 'INVSIB', 'TextMix', 'SentMix', 'WordMix']

In [4]:
MODEL_NAMES = ['bert-base-uncased', 'roberta-base', 'xlnet-base-cased']

In [5]:
use_pretrain = False

results = []
for MODEL_NAME in MODEL_NAMES:
    for t in ['ORIG', 'INV', 'SIB', 'INVSIB', 'TextMix', 'SentMix', 'WordMix']: 
        
        soft_target = False
        eval_only = False
        
        checkpoint = 'pretrained/' + MODEL_NAME + "-sst2-ORIG+" + t 
        tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
        
        if t == 'ORIG':
            train_dataset = load_dataset('glue', 'sst2', split='train[:90%]')
            train_dataset.rename_column_('sentence', 'text')
        else: 
            
            # load custom data    
            text = npy_load("./assets/SST2/" + t + "/text.npy")
            label = npy_load("./assets/SST2/" + t + "/label.npy")
            if len(label.shape) > 1:
                df = pd.DataFrame({'text': text, 'label': label.tolist()})
                df.text = df.text.astype(str)
                df.label = df.label.map(lambda y: np.array(y))
            else:
                df = pd.DataFrame({'text': text, 'label': label})
                df.text = df.text.astype(str)
                df.label = df.label.astype(object)
            train_dataset = Dataset.from_pandas(df) 
            
            # load orig data
            orig_dataset = load_dataset('glue', 'sst2', split='train[:90%]')
            orig_dataset.remove_columns_(['idx'])
            orig_dataset.rename_column_('sentence', 'text')
            df = orig_dataset.to_pandas()
            df = df[df.columns[::-1]]
            df.text = df.text.astype(str)
            if len(label.shape) > 1:
                df.label = df.label.map(one_hot_encode)
            else:
                df.label = df.label.astype(object)
            orig_dataset = Dataset.from_pandas(df)
            
            # merge orig + custom data
            train_dataset = concatenate_datasets([orig_dataset, train_dataset])
            train_dataset.shuffle()
            
        if use_pretrain and os.path.exists(checkpoint):
            print('loading {}...'.format(checkpoint))
            MODEL_NAME = checkpoint
            eval_only = True
            
        # split to get train
        dataset_dict = train_dataset.train_test_split(
            test_size = 0.05,
            train_size = 0.95,
            shuffle = True
        )
        train_dataset = dataset_dict['train']
        eval_dataset = dataset_dict['test']
        test_dataset = load_dataset('glue', 'sst2', split='train[-10%:]')
        test_dataset.rename_column_('sentence', 'text')
        
        model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME).to(device)
            
        train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
        eval_dataset = eval_dataset.map(tokenize, batched=True, batch_size=len(eval_dataset))
        test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))
        train_dataset.rename_column_('label', 'labels')
        eval_dataset.rename_column_('label', 'labels')
        test_dataset.rename_column_('label', 'labels')
        train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
        eval_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
        test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
        train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
        eval_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
        test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
        
        if len(np.array(train_dataset['labels']).shape) > 1:
            soft_target = True
        
        train_batch_size = 8
        eval_batch_size = 32
        num_epoch = 10
        gradient_accumulation_steps=1
        max_steps = int((len(train_dataset) * num_epoch / gradient_accumulation_steps) / train_batch_size)

        training_args = TrainingArguments(
            seed=1,
            # adafactor=True,
            output_dir=checkpoint,
            overwrite_output_dir=True,
            max_steps=max_steps,
            save_steps=int(max_steps / 10),
            save_total_limit=1,
            per_device_train_batch_size=train_batch_size,
            per_device_eval_batch_size=eval_batch_size,
            # gradient_accumulation_steps=gradient_accumulation_steps, 
            warmup_steps=int(max_steps / 10),
            weight_decay=0.01,
            logging_dir='./logs',
            logging_steps=2000,
            logging_first_step=True,
            load_best_model_at_end=True,
            metric_for_best_model="accuracy",
            greater_is_better=True,
            evaluation_strategy="steps",
            # run_name=checkpoint
        )

        if soft_target:
            trainer = Trainer_w_soft_target(
                model=model,
                args=training_args,
                compute_metrics=compute_metrics_w_soft_target,
                train_dataset=train_dataset,
                eval_dataset=eval_dataset,
                data_collator=DefaultCollator(),
                callbacks=[EarlyStoppingCallback(early_stopping_patience=10)]
            )
        else: 
            trainer = Trainer(
                model=model,
                args=training_args,
                compute_metrics=compute_metrics,
                train_dataset=train_dataset,
                eval_dataset=eval_dataset,
                callbacks=[EarlyStoppingCallback(early_stopping_patience=10)]
            )

        if not eval_only:
            trainer.train()
        
        trainer.compute_metrics = compute_metrics
            
        # test ORIG
        trainer.eval_dataset = test_dataset
        out = trainer.evaluate()
        out['run'] = checkpoint
        print('ORIG for {}\n{}'.format(checkpoint, out))   
        
        results.append(out)

Reusing dataset glue (C:\Users\sleev\.cache\huggingface\datasets\glue\sst2\1.0.0\7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4)
rename_column_ is deprecated and will be removed in the next major version of datasets. Use the dataset.rename_column method instead.
Loading cached split indices for dataset at C:\Users\sleev\.cache\huggingface\datasets\glue\sst2\1.0.0\7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4\cache-f091fffe09931764.arrow and C:\Users\sleev\.cache\huggingface\datasets\glue\sst2\1.0.0\7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4\cache-b8768bf7f4b494b6.arrow
Reusing dataset glue (C:\Users\sleev\.cache\huggingface\datasets\glue\sst2\1.0.0\7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4)
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.den

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Runtime,Samples Per Second
2000,0.414900,0.292297,0.913230,0.912701,0.911576,0.915729,7.599600,398.839000
4000,0.314300,0.282485,0.916859,0.914942,0.921683,0.911367,7.614300,398.066000
6000,0.294900,0.322373,0.925437,0.924978,0.923799,0.928048,7.602000,398.713000
8000,0.283800,0.332350,0.920488,0.918630,0.925671,0.914931,7.596200,399.015000
10000,0.275400,0.299632,0.931376,0.930746,0.929743,0.932152,7.591800,399.246000
12000,0.260900,0.264620,0.928736,0.928145,0.926992,0.929994,7.596900,398.980000
14000,0.253600,0.332025,0.931046,0.929848,0.932649,0.927892,7.590400,399.322000
16000,0.204200,0.297886,0.933025,0.932449,0.931331,0.934152,7.591000,399.290000
18000,0.219400,0.324828,0.925437,0.923869,0.929176,0.920784,7.588900,399.401000
20000,0.232500,0.283990,0.938964,0.938267,0.937938,0.938622,7.586700,399.516000


ORIG for pretrained/bert-base-uncased-sst2-ORIG+ORIG
{'eval_loss': 0.28216299414634705, 'eval_accuracy': 0.9383815887156645, 'eval_f1': 0.9373408328474395, 'eval_precision': 0.9382654159668381, 'eval_recall': 0.9365290144689284, 'eval_runtime': 16.9324, 'eval_samples_per_second': 397.758, 'epoch': 7.5, 'run': 'pretrained/bert-base-uncased-sst2-ORIG+ORIG'}


Reusing dataset glue (C:\Users\sleev\.cache\huggingface\datasets\glue\sst2\1.0.0\7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4)
remove_columns_ is deprecated and will be removed in the next major version of datasets. Use the dataset.remove_columns method instead.
Reusing dataset glue (C:\Users\sleev\.cache\huggingface\datasets\glue\sst2\1.0.0\7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4)
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (

Loading cached processed dataset at C:\Users\sleev\.cache\huggingface\datasets\glue\sst2\1.0.0\7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4\cache-58c5c95f3f3c7bcb.arrow
W&B installed but not logged in. Run `wandb login` or set the WANDB_API_KEY env variable.


...

ORIG for pretrained/xlnet-base-cased-sst2-ORIG+SentMix
{'eval_loss': 2.606091260910034, 'eval_accuracy': 0.9291759465478842, 'eval_f1': 0.9277320216590118, 'eval_precision': 0.9306033664763365, 'eval_recall': 0.9256503338708106, 'eval_runtime': 26.574, 'eval_samples_per_second': 253.443, 'epoch': 1.76, 'run': 'pretrained/xlnet-base-cased-sst2-ORIG+SentMix'}


Reusing dataset glue (C:\Users\sleev\.cache\huggingface\datasets\glue\sst2\1.0.0\7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4)
Reusing dataset glue (C:\Users\sleev\.cache\huggingface\datasets\glue\sst2\1.0.0\7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4)
Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not

Loading cached processed dataset at C:\Users\sleev\.cache\huggingface\datasets\glue\sst2\1.0.0\7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4\cache-0617f0b0800fe559.arrow
W&B installed but not logged in. Run `wandb login` or set the WANDB_API_KEY env variable.


Step,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
2000,0.432100,0.298179,0.596130,107.078400,113.216000
4000,0.319500,0.330469,0.575991,106.782600,113.530000
6000,0.300500,0.262498,0.622632,106.565000,113.762000
8000,0.288000,0.291122,0.628319,106.378700,113.961000
10000,0.275900,0.242179,0.632480,106.370400,113.970000
12000,0.272400,0.257062,0.643878,106.335100,114.007000
14000,0.261300,0.235138,0.645365,106.735600,113.580000
16000,0.255600,0.261517,0.643793,106.624400,113.698000
18000,0.248800,0.236509,0.638936,106.664100,113.656000
20000,0.252000,0.241187,0.644712,106.788300,113.524000


ORIG for pretrained/xlnet-base-cased-sst2-ORIG+WordMix
{'eval_loss': 2.8750319480895996, 'eval_accuracy': 0.9365998515219005, 'eval_f1': 0.9357062694840792, 'eval_precision': 0.9353618379608855, 'eval_recall': 0.9360734147043492, 'eval_runtime': 26.6857, 'eval_samples_per_second': 252.383, 'epoch': 1.53, 'run': 'pretrained/xlnet-base-cased-sst2-ORIG+WordMix'}


In [6]:
df = pd.DataFrame(results)
df

,eval_loss,eval_accuracy,eval_f1,eval_precision,eval_recall,eval_runtime,eval_samples_per_second,epoch,run
0,0.282163,0.938382,0.937341,0.938265,0.936529,16.9324,397.758,7.50,pretrained/bert-base-uncased-sst2-ORIG+ORIG
1,0.251824,0.925761,0.924765,0.924184,0.925420,17.0465,395.095,1.94,pretrained/bert-base-uncased-sst2-ORIG+INV
2,3.599298,0.943133,0.942239,0.942597,0.941900,17.0787,394.350,9.31,pretrained/bert-base-uncased-sst2-ORIG+SIB
3,2.637452,0.933482,0.932556,0.932146,0.933000,16.9467,397.422,4.17,pretrained/bert-base-uncased-sst2-ORIG+INVSIB
4,5.396307,0.947884,0.947161,0.946722,0.947636,16.9840,396.550,10.00,pretrained/bert-base-uncased-sst2-ORIG+TextMix
5,5.498501,0.949963,0.949276,0.948780,0.949819,16.9856,396.512,10.00,pretrained/bert-base-uncased-sst2-ORIG+SentMix
6,4.707505,0.945212,0.944412,0.944269,0.944558,16.9568,397.186,7.64,pretrained/bert-base-uncased-sst2-ORIG+WordMix
7,0.384818,0.910913,0.910330,0.909143,0.914591,16.5429,407.124,3.33,pretrained/roberta-base-sst2-ORIG+ORIG
8,0.283746,0.908686,0.906146,0.913948,0.902003,16.4950,408.305,1.81,pretrained/roberta-base-sst2-ORIG+INV
9,1.917380,0.912101,0.910557,0.911763,0.909545,16.5944,405.859,2.22,pretrained/roberta-base-sst2-ORIG+SIB


In [7]:
df.to_csv('train_SST2_r3.csv')

In [8]:
df.to_clipboard(excel=True)